In [1]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage import color
from skimage.morphology import binary_erosion, square
import cv2

In [2]:
# Import a few sample images
img1 = plt.imread('Extracted Characters/sample1.png')
img2 = plt.imread('Extracted Characters/sample2.png')
img3 = plt.imread('Extracted Characters/sample3.png')
print(f'img1.shape: {img1.shape}')
print(f'img2.shape: {img2.shape}')
print(f'img3.shape: {img3.shape}')

img1.shape: (350, 462, 3)
img2.shape: (292, 333, 3)
img3.shape: (356, 311, 3)


# Preprocessing

In [3]:
# Scale and pad

# Based on tutorial: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
def make_square(img, desired_size=256, fill_color=[255, 255, 255]):
    if img.dtype is not np.uint8:
        print(f'Converting to int before scaling...')
        img = (255*img).astype(np.uint8)
        
    scale_factor = desired_size/max(img.shape[0], img.shape[1])
    resized = cv2.resize(img, (int(scale_factor*img.shape[1]), int(scale_factor*img.shape[0])))
    new_size = resized.shape
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    out = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=fill_color)
    out = out/255.0
    return out

img1_square = make_square(img1)
img2_square = make_square(img2)
img3_square = make_square(img3)
print(f'img1_square.shape: {img1_square.shape}')
print(f'img2_square.shape: {img2_square.shape}')
print(f'img3_square.shape: {img3_square.shape}')

Converting to int before scaling...
Converting to int before scaling...
Converting to int before scaling...
img1_square.shape: (256, 256, 3)
img2_square.shape: (256, 256, 3)
img3_square.shape: (256, 256, 3)


In [4]:
# Binarize
def binarize(img, threshold=0.5, invert=True):
    # Convert to grayscale
    if len(img.shape) >= 3:
        img = color.rgb2gray(img)
    
    # Threshold
    out = np.zeros_like(img)
    if invert: # detect dark characters
        mask = img < threshold
    else: # detect light characters
        mask = img > threshold
    out[mask] = 1
    return out

img1_bin = binarize(img1_square)
img2_bin = binarize(img2_square)
img3_bin = binarize(img3_square)

In [5]:
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(4*num_cols, 4*num_rows))
for i, img in enumerate([img1, img2, img3]):
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'Sample {i+1}')
for i, img in enumerate([img1_square, img2_square, img3_square]):
    axes[1, i].imshow(img)
    axes[1, i].set_title(f'Scaled & padded {i+1}')
for i, img in enumerate([img1_bin, img2_bin, img3_bin]):
    axes[2, i].imshow(img, cmap='gray')
    axes[2, i].set_title(f'Binarized {i+1}')
    
plt.suptitle('Pre-processing Samples', fontsize='xx-large')
plt.tight_layout()
plt.savefig('PreprocessingSamples.png')

# Feature Extraction (Contour Analysis)

In [13]:
img1_bin.shape

(256, 256)

In [25]:
img1_bin.dtype

dtype('float64')

In [47]:
import numpy as np
import cv2 as cv
import matplotlib

im = cv.imread('test.jpg')
imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

print(f'im.shape: {im.shape}')
print(f'imgray.shape: {imgray.shape}')
print(f'imgray.dtype: {imgray.dtype}')
print(ret)
print(f'thresh.shape: {thresh.shape}')
print(f'thresh.dtype: {thresh.dtype}')
print(f'thresh.max() = {thresh.max()}')
print(type(contours))
print(f'# of contours: {len(contours)}')

dst = im.copy()
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours):
    cv.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.imshow(dst)
plt.title('Test image with labeled contours')
plt.savefig('ContourTest.png')

im.shape: (350, 462, 3)
imgray.shape: (350, 462)
imgray.dtype: uint8
127.0
thresh.shape: (350, 462)
thresh.dtype: uint8
thresh.max() = 255
<class 'list'>
# of contours: 9


In [112]:
# Extract contours
im = (img1_bin*255).astype(np.uint8)
contours, hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print(f'# of contours: {len(contours)}')
print('# of points in each contour:')
for cnt in contours:
    print(f'\t{len(cnt)} points')

dst = (img1_square*255).astype(np.uint8)
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours):
    cv2.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.figure(figsize=(4, 4))
plt.imshow(dst)
plt.title('Sample 1 with labeled contours')
plt.savefig('ContoursSample1.png')

# of contours: 5
# of points in each contour:
	1 points
	103 points
	97 points
	964 points
	4 points


Hmm since most `cv2` functions use uint8 images instead of float64 images, maybe I should rewrite all my code to be likewise.

In [113]:
# Throw out any contours with 10 or fewer points
contours_trimmed = [cnt for cnt in contours if len(cnt) > 10]
print(f'# of remaining contours: {len(contours_trimmed)}')
for cnt in contours_trimmed:
    print(f'\t{len(cnt)} points')

dst = (img1_square*255).astype(np.uint8)
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours_trimmed):
    print(len(cnt))
    cv2.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.figure(figsize=(4, 4))
plt.imshow(dst)
plt.title('Sample 1 with labeled contours (trimmed)')
plt.savefig('ContoursTrimmedSample1.png')

# of remaining contours: 3
	103 points
	97 points
	964 points
103
97
964


In [114]:
# I'm assuming the points in the contour are already ordered
# But let's plot the points in a line graph to make sure

x_list = [x[0][0] for x in contours_trimmed[0]]
y_list = [x[0][1] for x in contours_trimmed[0]]

plt.figure(figsize=(4, 4))
plt.plot(x_list)
plt.plot(y_list)
plt.plot(x_list, y_list)
plt.title('Ordered Points')
plt.savefig('OrderedPoints.png')

In [115]:
# Remove every other point (starting with first point)
contours_dashed = [cnt[1::2] for cnt in contours_trimmed]
print(f'# of remaining contours: {len(contours_trimmed)}')
for cnt in contours_dashed:
    print(f'\t{len(cnt)} points')

dst = (img1_square*255).astype(np.uint8)
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours_dashed):
    print(len(cnt))
    # We can't use drawContours because the contour is no longer contiguous
    # Instead, fill in the points manually
    for point in cnt:
        x, y = point[0]
        color = [255*x for x in cmap(i)][0:-1] # only take 3 color channels, ignore alpha
        dst[y, x, :] = color

plt.figure(figsize=(4, 4), dpi=300)
plt.imshow(dst)
plt.title('Sample 1 with dashed contours')
plt.savefig('ContoursDashedSample1.png')

# of remaining contours: 3
	51 points
	48 points
	482 points
51
48
482


In [117]:
# Compare dashed vs undashed
def draw_contours(img, contours):
    if img.dtype is not np.uint8:
        dst = (255*img).astype(np.uint8)
    else:
        dst = img.copy()
    cmap = matplotlib.cm.get_cmap('tab10')
    for i, cnt in enumerate(contours):
        for point in cnt:
            x, y = point[0]
            color = [255*x for x in cmap(i)][0:-1] # only take 3 color channels, ignore alpha
            dst[y, x, :] = color
    return dst

fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=300)
axes[0].imshow(draw_contours(img1_square, contours_trimmed))
axes[0].set_title('Undashed')
axes[1].imshow(draw_contours(img1_square, contours_dashed))
axes[1].set_title('Dashed')
plt.tight_layout()
plt.savefig('DashedVsUndashed.png')

In [120]:
# How about every 3rd point? Or every 4th point?
fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=300)
axes[0].imshow(draw_contours(img1_square, [cnt[1::3] for cnt in contours_trimmed]))
axes[0].set_title('Dashed (every 3rd point)')
axes[1].imshow(draw_contours(img1_square, [cnt[1::4] for cnt in contours_trimmed]))
axes[1].set_title('Dashed (every 4th point)')
plt.tight_layout()
plt.savefig('Dashed3rdor4th.png')

In [136]:
# Find sequence of angles in all the contours combined
thetaseq = []
for i, cnt in enumerate(contours_dashed):
    for j, point in enumerate(cnt):
        if j == 0:
            prevx, prevy = point[0]
        else:
            x, y = point[0]
            thetaseq.append(np.arctan2(y-prevy, x-prevx))
            prevx = x
            prevy = y

print(f'# thetas: {len(thetaseq)}')
print(f'max theta: {max(thetaseq)}')
print(f'min theta: {min(thetaseq)}')

# Calculate histogram of angles
bins = 20
hist = np.histogram(thetaseq, bins=bins, range=(-np.pi, np.pi))
plt.figure(figsize=(4, 4), dpi=300)
plt.hist(thetaseq, bins=bins, range=(-np.pi, np.pi))
plt.title('Histogram of angles')
plt.xlabel('Angle (radians)')
plt.ylabel('Frequency')
plt.savefig('Histogram.png')

# thetas: 578
max theta: 3.141592653589793
min theta: -2.677945044588987


In [10]:
# Repeat for a few more images

# Contour Statistics
Treating each images's histogram as a sample of the random variable $X$, estimate:
* Mean vector $\mu = E[X]$
* Covariance matrix $K = E[(X-\mu)(X-\mu)^T] $